In [109]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [110]:
df = pd.read_csv('Housing.csv')


In [111]:
df

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


In [112]:
df.describe()

,price,area,bedrooms,bathrooms,stories,parking
count,5.450000e+02,545.000000,545.000000,545.000000,545.000000,545.000000
mean,4.766729e+06,5150.541284,2.965138,1.286239,1.805505,0.693578
std,1.870440e+06,2170.141023,0.738064,0.502470,0.867492,0.861586
min,1.750000e+06,1650.000000,1.000000,1.000000,1.000000,0.000000
25%,3.430000e+06,3600.000000,2.000000,1.000000,1.000000,0.000000
50%,4.340000e+06,4600.000000,3.000000,1.000000,2.000000,0.000000
75%,5.740000e+06,6360.000000,3.000000,2.000000,2.000000,1.000000
max,1.330000e+07,16200.000000,6.000000,4.000000,4.000000,3.000000


In [204]:
#кореляція ціни та фітчерів area, bedrooms, bathrooms
print(df['price'].corr(df['area']))
print(df['price'].corr(df['bedrooms']))
print(df['price'].corr(df['bathrooms']))

print('\nКореляція розраховується для того, щоб виявити наскільки впливають задані фітчери на показник,\nякий ми плнуємо прогнозувати за допомогою лінійної регресії.\n\nЯкщо кореляція приближена до 0, це означає, що показники не корелю.ть між собою.' )

0.5359973457780801
0.3664940257738684
0.5175453394550115

Кореляція розраховується для того, щоб виявити наскільки впливають задані фітчери на показник,
який ми плнуємо прогнозувати за допомогою лінійної регресії.

Якщо кореляція приближена до 0, це означає, що показники не корелю.ть між собою.


In [181]:
#напишіть функцію гіпотези лінійної регресії у векторному вигляді;
#створіть функцію для обчислення функції втрат у векторному вигляді;
#реалізуйте один крок градієнтного спуску;
#знайдіть найкращі параметри вектора w для датасету прогнозуючу ціну на будинок залежно від площі, кількості ванних кімнат та кількості спалень;


def cost(y: np.array, h: np.array) -> float:
    return np.mean((h - y) ** 2) / 2



class CustomLinearRegression:
    def __init__(self, lr: float = 0.01, thr: float = 0.0001, itr: int = 1000, amount: int = 1):
        self.lr = lr
        self.thr = thr
        self.itr = itr
        self.w = np.array([x * 1. for x in range(amount)])

    def predict(self, X: np.ndarray) -> list:
        return X @ self.w

    def update_w(self, X: np.array, y: np.array):
        self.w -= self.lr / len(y) * X.T @ (self.predict(X) - y)

    def fit(self, X: np.array, y: np.array):
        #print('!!!!!!!!!', y)
        last_cost = 1000

        for i in range(self.itr):
            self.update_w(X, y)

            new_cost = cost(y, self.predict(X))
            if last_cost - new_cost < self.thr:
                #print(i)
                #print(f"w {self.w}")
                break
            last_cost = new_cost
        #print(f"w {self.w}")

In [182]:
X = df[['area', 'bathrooms', 'bedrooms']]
y = np.array((df['price'])/1000000)
X['area'] = X['area']/1000

C:\Users\olgapasichnyuk\AppData\Local\Temp\ipykernel_17684\3365183297.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['area'] = X['area']/1000


In [183]:
#самописний модуль обчислення ленійної регресії
custom_lr = CustomLinearRegression(amount=X.shape[1], itr=1000)
custom_lr.fit(X, y)
custom_lr.w

area         0.377248
bathrooms    0.908417
bedrooms     0.562263
dtype: float64

In [190]:
#обчислення ленійної регресії за допомогою модуля LinearRegression із sklearn.linear_model

lr = LinearRegression().fit(X, y)
lr.coef_

array([0.37876275, 1.3860495 , 0.40682003])

In [189]:
#знайдіть ці ж параметри за допомогою аналітичного рішення;

h = np.array(y)

X_inv = np.linalg.pinv(X)

w = X_inv.dot(h)

w = X_inv @ h

w

array([0.37244835, 1.37031315, 0.36897467])

In [195]:

print('{:^20}|{:^20}|{:^20}|{:^20}'.format('feature', 'custom_LR_method', 'sklearn_LR_method', 'analytic_method'))
print('-'*85)  
print('{:^20}|{:^20}|{:^20}|{:^20}'.format('area', custom_lr.w[0], lr.coef_[0], w[0]))
print('-'*85) 
print('{:^20}|{:^20}|{:^20}|{:^20}'.format('bathrooms', custom_lr.w[1], lr.coef_[1], w[1]))
print('-'*85) 
print('{:^20}|{:^20}|{:^20}|{:^20}'.format('bedrooms', custom_lr.w[2], lr.coef_[2], w[2])) 


      feature       |  custom_LR_method  | sklearn_LR_method  |  analytic_method   
-------------------------------------------------------------------------------------
        area        |0.37724831583745566 |0.37876275358510353 | 0.3724483516514232 
-------------------------------------------------------------------------------------
     bathrooms      | 0.908416619858503  | 1.3860494977909985 | 1.3703131459151185 
-------------------------------------------------------------------------------------
      bedrooms      | 0.5622634870748984 | 0.4068200337691825 | 0.3689746715042291 


In [197]:
print('Висновок\nПри пошуку вектору вагів різними способами я отпримала приблизно однакові значення.\nСамописний модуль домоміг розібратись в принципах роботи модуля LinearRegression із sklearn.linear_model.\nДля того, щоб оптимального обчислення вектору вагів, необхідно нормалізувати данні,\nтобто привести значення фітчерів до однакової розмірності.')

Висновок
При пошуку вектору вагів різними способами я отпримала приблизно однакові значення.
Самописний модуль домоміг розібратись в принципах роботи модуля LinearRegression із sklearn.linear_model.
Для того, щоб оптимального обчислення вектору вагів, необхідно нормалізувати данні,
тобто привести значення фітчерів до однакової розмірності.
